In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.optim import lr_scheduler

import numpy as np

import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
from torch.autograd import Variable
from PIL import Image
import requests 
import json


In [ ]:
data_dir = 'flowers'   
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])


train_image_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
test_image_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_image_datasets = datasets.ImageFolder(valid_dir, transform=validation_transforms)


train_dataloaders = torch.utils.data.DataLoader(train_image_datasets, batch_size = 64, shuffle=True)
test_dataloaders = torch.utils.data.DataLoader(test_image_datasets, batch_size = 64, shuffle=True)
valid_dataloaders = torch.utils.data.DataLoader(valid_image_datasets, batch_size = 64, shuffle=True)

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
architectures = {"vgg16":25088,
              "densenet121" : 1024,}

def nn_setup(architecture='vgg16',dropout=0.5, hidden_layer1 = 4000,lr = 0.001):
    
    
    if architecture == 'vgg16':
        model = models.vgg16(pretrained=True)        
    elif architecture == 'densenet121':
        model = models.densenet121(pretrained=True)
    else:
        print("Please choose a valid model")
        
    
        
    for param in model.parameters():
        param.requires_grad = False

    from collections import OrderedDict
        
    classifier = nn.Sequential(OrderedDict([
            ('dropout',nn.Dropout(dropout)),
            ('inputs', nn.Linear(architectures[architecture], hidden_layer1)),
            ('relu1', nn.ReLU()),
            ('hidden_layer1', nn.Linear(hidden_layer1, 800)),
            ('relu2',nn.ReLU()),
            ('hidden_layer2',nn.Linear(800,200)),
            ('relu3',nn.ReLU()),
            ('hidden_layer3',nn.Linear(200,102)),
            ('output', nn.LogSoftmax(dim=1))
                          ]))
        
        
    model.classifier = classifier
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr )
    model.cuda()
        
    return model , optimizer ,criterion 

    

model,optimizer,criterion = nn_setup('vgg16')

In [ ]:
def deep_learning(model, train_dataloaders, test_dataloaders, epochs, print_all, criterion, optimizer, device='cpu'):
    epochs = epochs
    print_all = print_all
    steps = 0
    
    if device == 'gpu':
        device = 'cuda'
    model.to(device)
    
    for e in range(epochs):
        running_loss = 0
        
        for ii, (inputs, labels) in enumerate(train_dataloaders):
            steps += 1
    
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
    
            if steps % print_all ==0:
                model.eval()
        
                with torch.no_grad():
                     test_loss, accuracy = validation(model, test_dataloaders, criterion, device)
            
                     print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_all),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(test_dataloaders)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(test_dataloaders)))
        
                running_loss = 0
            
                model.train()

In [ ]:
def main():
   

    
    parser = argparse.ArgumentParser(description='Train a new network on a data set')

    parser.add_argument('data_dir', type=str, \
        help='Location of Image Dataset (should include train, valid and test folders)')
    parser.add_argument('--save_dir', type=str, \
        help='Directory to save checkpoints')
    parser.add_argument('--architecture', type=str, \
        help='Models architeture. Default is vgg16. Densenet121 also available')
    parser.add_argument('--lr', type=float, \
        help='Learning rate. Default is 0.001')
    parser.add_argument('--hidden_units1', type=int, \
        help='Hidden units. Default is 4000')
    parser.add_argument('--epochs', type=int, \
        help='Number of epochs. Default is 3')
    parser.add_argument('--gpu', action='store_true', \
        help='Use GPU inference if available')
    parser.add_argument('--save_all', type=int, \
        help='Number of steps to present results. Default is 5')
    
    args, _ = parser.parse_known_args()

    data_dir = args.data_dir

    save_dir = './'
    if args.save_dir:
        save_dir = args.save_dir

    architecture = 'vgg16'
    if args.architecture:
        architecture = args.architecture

    lr = 0.001
    if args.lr:
        lr = args.lr

    hidden_units1 = 4000
    if args.hidden_units1:
        hidden_units1 = args.hidden_units1

    epochs = 3
    if args.epochs:
        epochs = args.epochs

    save_all = 5
    if args.save_all:
        save_all = args.save_all

    cuda = False
    if args.gpu:
        if torch.cuda.is_available():
            cuda = True
        else:
            print("Warning! GPU usage was set, however no GPU is available ")




    deep_learning(model, train_dataloaders, valid_dataloaders, epochs=epochs, criterion=criterion, optimizer=optimizer, device='cpu')





if __name__ == '__main__':
    main()